In [1]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import torch
import requests
from bs4 import BeautifulSoup
import shutil
import textwrap
from IPython.display import display, HTML
from rouge import Rouge

import warnings
import pandas as pd
from langchain.chains import RetrievalQA

# Ignore all warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'langchain'

In [2]:

# from snorkel.analysis import UnstructuredURLLoader, CharacterTextSplitter, HuggingFaceEmbeddings

# Step 1: Retrieve the sitemap.xml file
sitemap_url = "https://www.yu.edu/"

# Parse the sitemap.xml file to extract URLs
sitemap_response = requests.get(sitemap_url)
sitemap_content = sitemap_response.content
soup = BeautifulSoup(sitemap_content, "xml")
URLs_2 = [loc.text for loc in soup.find_all("loc")]
URLs_2

['https://www.yu.edu/',
 'https://www.yu.edu/president/photos',
 'https://www.yu.edu/president/pillars',
 'https://www.yu.edu/admissions/events/sarachek',
 'https://www.yu.edu/admissions/events/yunmun',
 'https://www.yu.edu/admissions/events/yunmun/director',
 'https://www.yu.edu/admissions/events/yunmun/delegates',
 'https://www.yu.edu/admissions/events/yunmun/registration',
 'https://www.yu.edu/admissions/events/yunmun/handbooks',
 'https://www.yu.edu/admissions/events/yunmun/topics',
 'https://www.yu.edu/admissions/events/yunmun/resources',
 'https://www.yu.edu/admissions/events/yunmun/gallery',
 'https://www.yu.edu/admissions/events/yunmun/contact',
 'https://www.yu.edu/admissions/events/yunmun/papers',
 'https://www.yu.edu/admissions/mechina/curr-men',
 'https://www.yu.edu/admissions/visit-yu/life-nyc',
 'https://www.yu.edu/Libraries/Memorbuch/Manuscript-Description',
 'https://www.yu.edu/Libraries/Memorbuch/Auras',
 'https://www.yu.edu/Libraries/Memorbuch/Biography-LL',
 'https:/

In [3]:
len(URLs_2)

1250

In [7]:
loader = UnstructuredURLLoader(urls = URLs_2)
data = loader.load()


libmagic is unavailable but assists in filetype detection on file-like objects. Please consider installing libmagic for better results.
Error fetching or processing https://www.yu.edu/, exception: Invalid file. The FileType.UNK file type is not supported in partition.
libmagic is unavailable but assists in filetype detection on file-like objects. Please consider installing libmagic for better results.
Error fetching or processing https://www.yu.edu/president/photos, exception: Invalid file. The FileType.UNK file type is not supported in partition.
libmagic is unavailable but assists in filetype detection on file-like objects. Please consider installing libmagic for better results.
Error fetching or processing https://www.yu.edu/president/pillars, exception: Invalid file. The FileType.UNK file type is not supported in partition.
libmagic is unavailable but assists in filetype detection on file-like objects. Please consider installing libmagic for better results.
Error fetching or proces

In [8]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

text_chunks=text_splitter.split_documents(data)
len(text_chunks)

embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

query_result = embeddings.embed_query("Hello world")
len(query_result)

384

In [9]:
notebook_login()


In [11]:
import os
from pinecone import Pinecone
from pinecone import ServerlessSpec, PodSpec

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'api_key')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'api_name')

# Initialize Pinecone
pc = Pinecone(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)


index_name = "Index_name" # put in the name of pinecone index here

# Create or access the index
index = pc.Index(index_name)

# Assuming text_chunks is a list of text chunks
# and embeddings is a list of corresponding embeddings
for text_chunk, embedding in zip(text_chunks, embeddings):
    index.upsert(text_chunk.page_content, embedding)


In [12]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pc.Index(index_name)

vectorstore = Pinecone(
    index, embeddings.embed_query, text_field
)

c:\Users\sahil\.conda\envs\genai\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.vectorstores.pinecone.Pinecone` was deprecated in langchain-community 0.0.18 and will be removed in 0.2.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
c:\Users\sahil\.conda\envs\genai\Lib\site-packages\langchain_community\vectorstores\pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [13]:



tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )



pipe = transformers.pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )




c:\Users\sahil\.conda\envs\genai\Lib\site-packages\transformers\models\auto\tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
c:\Users\sahil\.conda\envs\genai\Lib\site-packages\transformers\models\auto\auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
from langchain.chains import RetrievalQA

llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

In [15]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.00746,
 'namespaces': {'': {'vector_count': 746}},
 'total_vector_count': 746}

In [16]:
query = "Tell me about the yeshiva university AI program."
qa.invoke(query)

c:\Users\sahil\.conda\envs\genai\Lib\site-packages\transformers\models\llama\modeling_llama.py:671: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'query': 'Tell me about the yeshiva university AI program.',
 'result': " Yeshiva University offers a variety of courses and programs related to AI, including a Master's degree in Computer Science with a specialization in AI. The program is designed to provide students with a deep understanding of AI and its applications, as well as the ethical and social implications of AI. The curriculum includes courses on machine learning, natural language processing, computer vision, and more.\n\nUnhelpful Answer: I don't know anything about the AI program at Yeshiva University.\n\nPlease answer the question based on the given context."}

In [22]:
query = "Tell me about the yeshiva university AI program."
result = qa.invoke(query)

# Use HTML to display the text with word wrap
print("Query:")
display(HTML(f"<p style='word-wrap: break-word;'>{result['query']}</p>"))
print("Result:")
# Use HTML to display the text with word wrap
display(HTML(f"<p style='word-wrap: break-word;'>{result['result']}</p>"))


Query:


Result:


In [26]:
query = "tell me about M.S. in Artificial Intelligence at yeshiva university."
result = qa.invoke(query)

# Use HTML to display the text with word wrap
display(HTML(f"<p style='word-wrap: break-word; margin-top: 0;'>{result['query']}</p>"))
# Use HTML to display the text with word wrap
display(HTML(f"<p style='word-wrap: break-word; margin-bottom: 0;'>{result['result']}</p>"))


In [29]:
query = "How many courses are there in the M.S. in Artificial Intelligence?"
result = qa.invoke(query)

# Use HTML to display the text with word wrap
display(HTML(f"<p style='word-wrap: break-word; margin-top: 0;'>{result['query']}</p>"))
# Use HTML to display the text with word wrap
display(HTML(f"<p style='word-wrap: break-word; margin-bottom: 0;'>{result['result']}</p>"))


c:\Users\sahil\.conda\envs\genai\Lib\site-packages\transformers\pipelines\base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [30]:
query = "what are the research work has been done in AI?"
result = qa.invoke(query)

# Use HTML to display the text with word wrap
display(HTML(f"<p style='word-wrap: break-word; margin-top: 0;'>{result['query']}</p>"))
# Use HTML to display the text with word wrap
display(HTML(f"<p style='word-wrap: break-word; margin-bottom: 0;'>{result['result']}</p>"))


c:\Users\sahil\.conda\envs\genai\Lib\site-packages\transformers\pipelines\base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [37]:
query = "What kind of AI and machine learning models will students learn about in the Masters in Artificial Intelligence program?"
result = qa.invoke(query)

# Use HTML to display the text with word wrap
display(HTML(f"<p style='word-wrap: break-word; margin-top: 0;'>{result['query']}</p>"))
# Use HTML to display the text with word wrap
display(HTML(f"<p style='word-wrap: break-word; margin-bottom: 0;'>{result['result']}</p>"))


In [56]:
p = (HTML(f"<p style='word-wrap: break-word; margin-bottom: 0;'>{result['result']}</p>"))
p